In [1]:
import sys
sys.version

'2.7.11 |Anaconda 2.4.1 (x86_64)| (default, Dec  6 2015, 18:57:58) \n[GCC 4.2.1 (Apple Inc. build 5577)]'

In [2]:
import time
import numpy as np
import math
import pandas as pd
import itertools

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

%matplotlib inline
import matplotlib.pyplot as plt

<div class="alert alert-info">
<strong>LOAD DATA</strong>
</div>

In [3]:
# load Y values
df_Y = pd.read_csv('data/train_Y.csv')
df_Y = df_Y.drop(['Unnamed: 0'], axis=1)
Y_train = df_Y.GAP.values
test_idx = Y_train.shape[0]
test_idx

(1000000,)

In [4]:
# load 1-char features
df_1char = pd.read_csv('data/enhaced_features_1char.csv')
df_1char = df_1char.drop(['Unnamed: 0'], axis=1)
df_1char.shape

(1824230, 52)

In [5]:
# load 2-char features
df_2char = pd.read_csv('data/enhaced_features_2char.csv')
df_2char = df_2char.drop(['Unnamed: 0'], axis=1)
df_2char.shape

(1824230, 190)

In [6]:
# merge and drop duplicate columns
cols_to_use = df_2char.columns.difference(df_1char.columns)
df_comb = pd.concat([df_1char, df_2char[cols_to_use]], axis=1)
df_comb.shape

(1824230, 211)

In [7]:
# clear memory before moving on
df_1char = None
df_2char = None

In [8]:
# load 3-char features
df_3char = pd.read_csv('data/enhaced_features_3char.csv')
df_3char = df_3char.drop(['Unnamed: 0'], axis=1)
df_3char.shape

(1824230, 593)

In [9]:
# merge and drop duplicate columns
cols_to_use = df_3char.columns.difference(df_comb.columns)
df_comb = pd.concat([df_comb, df_3char[cols_to_use]], axis=1)
df_comb.shape

(1824230, 773)

In [10]:
# clear memory before moving on
df_3char = None

In [11]:
# load three bond type features
df_bondtypes = pd.read_csv('data/bond_type_counts.csv')
df_bondtypes = df_bondtypes.drop(['Unnamed: 0'], axis=1)
df_bondtypes.shape

(1824230, 3)

In [12]:
# combine all data
df_comb = pd.concat([df_comb, df_bondtypes], axis=1)
df_comb.shape

(1824230, 776)

In [13]:
# clear memory before moving on
df_bondtypes = None

In [18]:
# split back up into training and test data
lcols = df_comb.columns.values.tolist()
vals = df_comb.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]

print 'Train features:', X_train.shape
print 'Train gap:', Y_train.shape
print 'Test features:', X_test.shape

Train features: (1000000, 776)
Train gap: (1000000,)
Test features: (824230, 776)


<div class="alert alert-info">
<strong>PREDICTORS</strong>
</div>

In [19]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

**Linear Regression**

In [20]:
# # linear regression - test & validation split
# LR = LinearRegression()
# LR.fit(X_train[:800000], Y_train[:800000])
# LR_rmse_train = math.sqrt(mean_squared_error(Y_train[:800000], LR.predict(X_train[:800000])))
# LR_rmse_val = math.sqrt(mean_squared_error(Y_train[800000:], LR.predict(X_train[800000:])))
# print 'Linear regression RMSE - training set = %0.5f' % LR_rmse_train
# print 'Linear regression RMSE - validation set = %0.5f' % LR_rmse_val
# print 'Baseline linear regression RMSE = 0.29892'

In [21]:
# # linear regression - training & test split
# LR = LinearRegression()
# LR.fit(X_train, Y_train)
# LR_pred = LR.predict(X_test)
# LR_rmse = math.sqrt(mean_squared_error(Y_train, LR.predict(X_train)))
# print 'New linear regression RMSE = %0.5f' % LR_rmse
# print 'Baseline linear regression RMSE = 0.29892'
# write_to_file('data/LR_3char.csv_bondtypes.csv', LR_pred)

**Random Forest**

In [22]:
# random forest regressor - test & validation split
RF = RandomForestRegressor()
RF.fit(X_train[:800000], Y_train[:800000])
RF_rmse_train = math.sqrt(mean_squared_error(Y_train[:800000], RF.predict(X_train[:800000])))
RF_rmse_val = math.sqrt(mean_squared_error(Y_train[800000:], RF.predict(X_train[800000:])))
print 'Random forest RMSE - training set = %0.5f' % RF_rmse_train
print 'Random forest RMSE - validation set = %0.5f' % RF_rmse_val
print 'Baseline random forest RMSE = 0.27188'

Random forest RMSE - training set = 0.03285
Random forest RMSE - validation set = 0.07156
Baseline random forest RMSE = 0.27188


In [24]:
# # random forest regressor - training & test split
# RF = RandomForestRegressor()
# RF.fit(X_train, Y_train)
# RF_pred = RF.predict(X_test)
# RF_rmse = math.sqrt(mean_squared_error(Y_train, RF.predict(X_train)))
# print 'New random forest RMSE = %0.5f' % RF_rmse
# print 'Baseline random forest RMSE = 0.27188'
# write_to_file('data/RF_1_2_3char_bondtypes.csv', RF_pred)